In [11]:
from PIL import Image
import cv2
import os
import random
from tqdm import tqdm
import dlib
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dir = r"/content/drive/MyDrive/Dataset/train"

In [5]:
# Output Path for data
output_path = "/content/drive/MyDrive/skin_detection"

In [6]:
os.makedirs(output_path, exist_ok=True)

In [7]:
def detect_face(image):
    # Load the pre-trained HOG face detector from dlib
    face_detector = dlib.get_frontal_face_detector()

    # Detect faces in the image
    faces = face_detector(image, 1)

    if len(faces) > 0:
        # Assuming only one face, you can modify this if there are multiple faces
        x, y, w, h = faces[0].left(), faces[0].top(), faces[0].width(), faces[0].height()
        bounding_box = (x, y, w, h)
        return bounding_box
    else:
        # If no face is detected, use the entire image as the bounding box
        height, width, _ = image.shape
        bounding_box = (0, 0, width, height)
        return bounding_box

In [21]:
def extract_skin_roi(image, bounding_box):
    x, y, w, h = bounding_box

    # Extract the face region
    face_roi = image[y:y + h, x:x + w]

    # Check if the face_roi is empty or too small
    if face_roi is None or face_roi.size == 0 or face_roi.shape[0] < 10 or face_roi.shape[1] < 10:
        # print("Warning: Empty or small face_roi. Using the entire image as ROI.")
        return image

    # Convert the face region to HSV color space
    hsv_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)

    # Define a range for skin color in HSV
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Create a binary mask for the skin color range
    skin_mask = cv2.inRange(hsv_face, lower_skin, upper_skin)

    # Apply the mask to extract the skin region
    skin_roi = cv2.bitwise_and(face_roi, face_roi, mask=skin_mask)

    return skin_roi


In [29]:
def process_image(image_path, class_name, output_path):
    # Load the image
    img = cv2.imread(image_path)

    # Detect face
    bounding_box = detect_face(img)

    # Extract skin ROI
    skin_roi = extract_skin_roi(img, bounding_box)

    # Resize to a standard size (248x248)
    skin_roi_resized = cv2.resize(skin_roi, (248, 248))

    # Save the processed image in the respective class folder
    # save_path = os.path.join(output_path, class_name, os.path.basename(image_path))
    print(f"Saving at: {output_path}")

    # Save the image using cv2.imwrite
    cv2.imwrite(output_path, cv2.cvtColor(skin_roi_resized, cv2.COLOR_BGR2RGB))


In [31]:
# 80% of data is selected for training and 20% for validating
validation_size = 0.2

# Create training and validating classes
classes = ['Combination', 'Dry', 'Normal', 'Oily']
for class_name in classes:
    os.makedirs(os.path.join(output_path, "train", class_name), exist_ok=True)
    os.makedirs(os.path.join(output_path, "validation", class_name), exist_ok=True)

# Get the list of all image files in the dataset folder for each class
all_images = {class_name: [f for f in os.listdir(os.path.join(dir, class_name)) if f.lower().endswith('.jpg')] for class_name in classes}

# Shuffle the lists of image files for each class
for class_name in classes:
    random.shuffle(all_images[class_name])

# Calculate the number of images to use for the validation set for each class
num_validation = {class_name: int(len(all_images[class_name]) * validation_size) for class_name in classes}

# Move the images to the appropriate train and validation folders for each class
for class_name in classes:
    for i, image_name in tqdm(enumerate(all_images[class_name]), desc=f"Processing {class_name} images"):
        src_path = os.path.join(dir, class_name, image_name)
        if i < num_validation[class_name]:
            dst_path = os.path.join(output_path, "validation", class_name, image_name)
        else:
            dst_path = os.path.join(output_path, "train", class_name, image_name)

        # Process the image and save
        process_image(src_path, class_name, dst_path)

print("Data processing completed.")

Processing Combination images: 1it [00:00,  1.24it/s]

Saving at: /content/drive/MyDrive/skin_detection/validation/Combination/combination_-_b95e08ce7eb385ee6d58_jpg.rf.458badecd96f012a85e5e4d4974c1348.jpg


Processing Combination images: 2it [00:01,  1.21it/s]

Saving at: /content/drive/MyDrive/skin_detection/validation/Combination/combination_-_d4d3369474d0907f660d_jpg.rf.fea3267afe992118a97ab96eb68e3105.jpg


Processing Combination images: 3it [00:02,  1.26it/s]

...

Saving at: /content/drive/MyDrive/skin_detection/validation/Combination/combination_-_aaecf6fbe61e51dec18a_jpg.rf.bf91b88c9bac27753f6c8b4a2f02e481.jpg


Processing Combination images: 34it [00:22,  1.61it/s]

Saving at: /content/drive/MyDrive/skin_detection/validation/Combination/combination_-_65ab6e135242c408e342_jpg.rf.3d57dd1eccfd16355267872e76a32c0d.jpg


Processing Combination images: 34it [00:23,  1.46it/s]


KeyboardInterrupt: 

In [19]:
train_dir = os.path.join(output_path, 'train')
validation_dir = os.path.join(output_path, 'validation')

# Directory with training images
train_combination_dir = os.path.join(train_dir, 'Combination')
train_dry_dir = os.path.join(train_dir, 'Dry')
train_normal_dir = os.path.join(train_dir, 'Normal')
train_oily_dir = os.path.join(train_dir, 'Oily')


# Directory with validation cat/dog pictures
validation_combination_dir = os.path.join(validation_dir, 'Combination')
validation_dry_dir = os.path.join(validation_dir, 'Dry')
validation_normal_dir = os.path.join(validation_dir, 'Normal')
validation_oily_dir = os.path.join(validation_dir, 'Oily')

In [20]:
def count_images_in_folder(folder_path):
    return sum(1 for file in os.listdir(folder_path) if file.endswith(('.jpg', '.png')))

# Define your directory paths
directories = {
    'Train combination skin Images': train_combination_dir,
    'Train dry skin Images': train_dry_dir,
    'Train normal skin Images': train_normal_dir,
    'Train oily skin Images': train_oily_dir,
    '\nValidation combination skin Images': validation_combination_dir,
    'Validation dry skin Images': validation_dry_dir,
    'Validation normal skin Images': validation_normal_dir,
    'Validation oily skin Images': validation_oily_dir
}

# Count and print the image counts for each directory
for directory_name, directory_path in directories.items():
    image_count = count_images_in_folder(directory_path)
    print(f"{directory_name}: {image_count}")

Train combination skin Images: 0
Train dry skin Images: 0
Train normal skin Images: 0
Train oily skin Images: 0

Validation combination skin Images: 0
Validation dry skin Images: 0
Validation normal skin Images: 0
Validation oily skin Images: 0
